In [ ]:
!unzip -q rice-dataset-sample.zip

In [ ]:
!pip install split-folders

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Dispositivo disponible: {device}")

## División de los conjuntos train, val y test


In [ ]:
import splitfolders

input_dir = '/content/rice-dataset-sample'
output_dir = '/content/rice-dataset-sample-splits'

splitfolders.ratio(
    input_dir,
    output=output_dir,
    seed=42,
    ratio=(.7, .2, .1)
)

Copying files: 50 files [00:00, 5164.00 files/s]


## Instanciar DataLoaders

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

In [ ]:
from torchvision import datasets

dataset_path = '/content/rice-dataset-sample-splits'

train_dataset = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{dataset_path}/val", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{dataset_path}/test", transform=transform)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 16
shuffle = True

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

## Primera red convolucional

In [ ]:
 import torch.nn as nn
 import torch.nn.functional as F

 class SimpleConvNet(nn.Module):
  def __init__(self):
    super(SimpleConvNet, self).__init__()
    # TODO: Crea una capa convolucional de 2 dimensiones con los parámetros:
    # - in_channels = 3
    # - out_channels = 16
    # - kernel_size = 3
    # - padding = 1
    # TIP: Consulta la documentación de la función Conv2d
    self.conv = nn.Conv2d(
        in_channels=3,
        out_channels=16,
        kernel_size=3,
        padding=1
    )
    # TODO: Crea una capa de maxpool de 2 dimensiones con los parámetros:
    # - kernel_size = 2
    # - stride = 2
    # TIP: Consulta la documentación de la función MaxPool2d
    self.pool = nn.MaxPool2d(
        kernel_size=2,
        stride=2
    )
    self.fc1 = nn.Linear(16 * 14 * 14, 120)
    self.fc2 = nn.Linear(120, 5)

  def forward(self, x):
    output = self.conv(x)
    output = F.relu(output)
    output = self.pool(output)
    output = torch.flatten(output, 1)
    output = self.fc1(output)
    output = F.relu(output)
    output = self.fc2(output)

    return output

In [ ]:
model = SimpleConvNet().to(device)
model

SimpleConvNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=5, bias=True)
)

## Entrenar la red convolucional

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
  running_loss = 0.0
  for inputs, labels in train_dataloader:
    # TODO: Envía los inputs y las labels al device adecuado
    inputs, labels = inputs.to(device), labels.to(device)

    # TODO: Deja a cero los gradientes del opitmizador
    # TIPO: Consulta la función zero_grad
    optimizer.zero_grad()

    # TODO: Calcula el output del modelo
    outputs = model(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch: {epoch+1} Loss: {running_loss/len(train_dataloader)}")

Epoch: 1 Loss: 1.6498209238052368
Epoch: 2 Loss: 1.583358883857727
Epoch: 3 Loss: 1.496825893719991
Epoch: 4 Loss: 1.4010985294977825
Epoch: 5 Loss: 1.3505812486012776


## Evaluar la red convolucional

In [ ]:
from sklearn.metrics import classification_report

model.eval()

with torch.no_grad():
  all_preds = []
  all_labels = []

  for inputs, labels in test_dataloader:
    inputs, lagbels = inputs.to(device), labels.to(device)

    # TODO: Calcula la salida del modelo
    outputs = model(inputs)

    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.numpy(force=True))
    all_labels.extend(labels.numpy(force=True))

print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

              precision    recall  f1-score   support

     Arborio       0.50      1.00      0.67         1
     Basmati       0.00      0.00      0.00         1
      Ipsala       0.50      1.00      0.67         1
     Jasmine       0.00      0.00      0.00         1
   Karacadag       0.00      0.00      0.00         1

    accuracy                           0.40         5
   macro avg       0.20      0.40      0.27         5
weighted avg       0.20      0.40      0.27         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
